In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys
os.environ["GDAL_DATA"] = "/srv/conda/envs/notebook/share/gdal" # need to specify to make gdal work
os.environ["PROJ_LIB"] = "/srv/conda/envs/notebook/share/proj" # need to specify to make pyproj work
os.environ["PROJ_DATA"] = "/srv/conda/envs/notebook/share/proj" # need to specify to make pyproj work
import numpy as np
import pandas as pd
from matplotlib import cm 
import matplotlib
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
import geopandas as gpd
from ipyleaflet import Map, basemaps, Polygon, Polyline, GeoData, LayersControl
from datetime import datetime
from dateutil.relativedelta import relativedelta
from rasterio import warp
import icepyx as ipx
import shutil

if 'cmcrameri' not in sys.modules:
    ! pip3 install cmcrameri >/dev/null
if 'contextily' not in sys.modules:
    ! pip3 install contextily >/dev/null
import cmcrameri.cm as cmc
import contextily as cx

try:
    from ed.edcreds import getedcreds
except:
    print('No earthdata credentials found.')
    print('To set up, rename the folder "ed_example" to "ed".')
    print('Then, add your earthdata credentials to "ed/edcreds.py".')

from utils.nsidc import download_is2
from utils.readers import read_atl06
from utils.S2 import plotS2cloudfree, add_inset

## Show area of interest on a map

In [ ]:
# I just drew this randomly on a map...
shape = 'data/shapefiles/amery_gillock_kreitzer.shp'

#################################################
# for testing
# shape = 'data/shapefiles/mini_test_region.shp'
#################################################

gdf = gpd.read_file(shape)
geom = [i for i in gdf.geometry]
lons,lats = geom[0].exterior.coords.xy
shape_coords = list(zip(lats,lons))
shape_bounds = [[np.min(lats),np.min(lons)], [np.max(lats),np.max(lons)]]

m=Map(basemap=basemaps.Esri.WorldImagery,center=[np.mean(lats),np.mean(lons)],zoom=7)
m.fit_bounds(shape_bounds)
polygon = Polygon(locations=shape_coords, color="red", fill_color="red", name='polygon_buffered')
m.add_layer(polygon)
m.add_control(LayersControl())
m

## Download the ATL06 ICESat-2 Data

In [ ]:
os.environ["EARTHDATA_USERNAME"] = getedcreds()[0]
os.environ["EARTHDATA_PASSWORD"] = getedcreds()[1]

short_name = 'ATL06'
spatial_extent = shape
temporal = ['2018-01-01','2030-01-01']

#################################################
# for testing
# temporal = ['2020-01-01','2020-03-01']
#################################################

region_a = ipx.Query(short_name, spatial_extent, temporal)
results = region_a.avail_granules(ids=True)
print('Found granules:')
results_print = np.array(results).flatten()
for i in range(np.min((len(results_print),10))):
    print(results_print[i])
if len(results_print)>10:
    print('...and %d more results.' % (len(results_print)-10))
region_a.avail_granules(ids=True)
output_dir = 'data/IS2/' + short_name + '_' + shape[:shape.rfind('.')].split('/')[-1]

#### uncomment to download again
region_a.download_granules(output_dir)
print('\nsaved files to %s' % output_dir)

### Get a list of all the ATL06 data files over this region

In [ ]:
search_for = 'ATL06_'
search_in = output_dir + '/'
filelist = [search_in+f for f in os.listdir(search_in) \
            if os.path.isfile(os.path.join(search_in, f)) & (search_for in f) & ('.h5' in f)]
filelist.sort()
print('There are %i files.' % len(filelist))

## get a dataframe with basic file info (for all files / all tracks)

In [ ]:
df_files = pd.DataFrame({'filename': filelist})
df_files['granule_id'] = df_files.apply(lambda x: x.filename[x.filename.find('ATL06_'):], axis=1)
df_files['date'] = df_files.apply(lambda x: x.granule_id[6:10]+'-'+x.granule_id[10:12]+'-'+x.granule_id[12:14], axis=1)
df_files['track'] = df_files.apply(lambda x: int(x.granule_id[21:25]), axis=1)

# figure out how many files there are per track 
files_per_track = df_files.groupby('track')[['filename']].count().rename(columns={'filename': 'number_of_files'})

# take only the tracks with at least min_files files 
min_files = 3
files_per_track = files_per_track[files_per_track.number_of_files >= min_files]

# show the results
files_per_track

## pick a single track / GTX combination, and work with only that from here on
Just pick a single combination to show some repeat tracks and develop code. If you are happy with your code and want to do this for more data in an automated way you can always put all your code into a loop that goes over all track/GTX combinations (if you have time for that). 

In [ ]:
# can always go back to here and change this manually to a different track/GTX by changing the values below
# then re-run all the following cells
track = 721
gtx = 'gt3l'

# print to confirm the track/GTX selection
print('SELECTION: track %s, %s' % (track, gtx))

# Figure out which files there are for this track --> get a dataframe with file info for just the selected track
df_thistrack = df_files[df_files.track == track].copy()
# sort by date and re-set index 
df_thistrack = df_thistrack.sort_values(by='date').reset_index(drop=True)
# store the number of files for this track (this is the number of repeat passes / distinct dates from which we have data)
nfiles = len(df_thistrack)
print('  number of files available:', nfiles)

# initialize data dictionary for all the data for this track/GTX combination
gtx_data_dict = {}

# LOOP FOR ALL FILES
for i in range(nfiles):

    # get the filename
    this_filename = df_thistrack.iloc[i].filename
    print('  - %d: %s ' % (i, this_filename))

    # read in the data from this file using the function read_atl06()
    ancillary, dfs = read_atl06(this_filename)

    # make sure that there is data for the selected GTX in the file that we have read in
    if gtx in dfs.keys():

        # select the dataframe with the data for our GTX from the "dfs" dictionary
        this_data = dfs[gtx]

        # set data with questionable quality summary values to nan (i.e. nodata value)
        this_data[this_data.qual_summary != 0] = np.nan

        # set data with elevation values > 100 km to nan (because that's unreasonable)
        this_data[this_data.h > 1e5] = np.nan

        # now, just save the data for this gtx along with the ancillary data
        ancillary['data'] = this_data
        ancillary['gtx'] = gtx

        # save the data to the gtx_data_dict, with the key being the date of the data acquisition
        gtx_data_dict[ancillary['date']] = ancillary

        # print a success message
        print('    --> got %s data for %s' % (gtx, ancillary['date']))

    else:
        # if there is no data for the selected gtx on that date do nothing but print a message
        print('    --> NO %s data available for %s' % (gtx, ancillary['date']))

# figure out how many dates have data for the selected track / GTX combination and print
number_of_dates = len(gtx_data_dict)
print('\nFinished reading in data for track %s, %s. There is data for %d dates.' % (track, gtx, number_of_dates))

## make a function for basic plotting

In [ ]:
def plot_repeat_track(gtx_data_dict, track, gtx):
    
    # make the figure for this track / GTX selection
    fig, ax = plt.subplots(figsize=[8,5])
    
    # loop through all dates
    for date in gtx_data_dict.keys():
    
        # get the data and ancillary for this date
        date_dict = gtx_data_dict[date]
    
        # get the strength of the beam (just an example of a property that you can access...)
        beamstrength = date_dict['gtx_strength_dict'][gtx]
    
        # get the elevation data for this date (we stored this in 'data' in the cell above)
        this_data = date_dict['data']
    
        # get the variables to plot (xatc and h)
        x_along_track = (this_data.xatc - this_data.xatc.min()) / 1000
        elevation = this_data.h
    
        # plot those variables
        ax.plot(x_along_track, elevation, label='%s, %s' % (date, beamstrength))
    
        # set the title and axis labels, set title to track/GTX selection that we made
        ax.set_title('track %s, %s' % (track, gtx))
        ax.set_xlabel('along-track distance (km)')
        ax.set_ylabel('elevation (m)')
    
        # make a legend
        ax.legend(loc='upper right', fontsize=7)

## call the function to plot elevation vs along-track distance for each date available

In [ ]:
plot_repeat_track(gtx_data_dict, track, gtx)

## now that you have all the data you could make any desired changes, and then plot again!
for example, see what happens when you run this code and then plot in the cell below again

In [ ]:
# make a copy of the data dictionary (to keep the old one so you don't have to re-run the cells above this)
new_gtx_data_dict = gtx_data_dict.copy()

# get mean across track distances from the data (initialize empty list, then fill with values for all dates)
mean_across_track_distances = []
for date in new_gtx_data_dict.keys():
    mean_across_track = np.nanmean(new_gtx_data_dict[date]['data'].yatc)
    new_gtx_data_dict[date]['mean_across_track'] = mean_across_track
    mean_across_track_distances.append(mean_across_track)

# get the median across-track distance for all repeat passes 
across_track_median = np.nanmedian(mean_across_track_distances)

# now delete the data that is further away from the median than max_difference_across_track meters
max_difference_across_track = 10 # meters
for date in list(new_gtx_data_dict.keys()):

    # if mean across-track distance from this date is less than max_difference_across_track meters away from the median for all dates
    across_track_offset = np.abs(new_gtx_data_dict[date]['mean_across_track'] - across_track_median)
    keep = np.abs(across_track_offset) <= max_difference_across_track
    action = 'keep' if keep else 'delete'
    print('%s: across-track offset from median = %8.2f m --> %s!' % (date, across_track_offset, action))
    
    if not keep:
        # delete the entry for this date
        del new_gtx_data_dict[date]

# call the plotting function again
plot_repeat_track(new_gtx_data_dict, track, gtx)